In [8]:
#Import modules and read in positions file; keep only useful columns
import json
import pandas as pd
from datetime import datetime, timedelta
import time
from yahoo_fin.stock_info import *

ticker_file = pd.read_json('./final_output.json',encoding='utf8')
ticker_file = ticker_file[['author','created_utc','regexed_combined']]

In [9]:
#Filter out rows without positions or with 'NONE ticker'; parse position date string into a datetime obj
ticker_file = ticker_file.loc[ticker_file['regexed_combined'] != 0]
for posted_positions in ticker_file['regexed_combined']:
    for position in posted_positions:
        position[3] = datetime.strftime(datetime.fromtimestamp(float(position[3]/1000)), '%Y-%m-%d-%w')
    
ticker_file = ticker_file[ticker_file['regexed_combined'].str[0].str[0] != 'NONE']

In [10]:
#parse created_utc into datetime object, replace column
datetimes = []
for timestamp in ticker_file['created_utc']:
    datetimes.append(datetime.strftime(datetime.fromtimestamp(timestamp), '%Y-%m-%d-%w'))
ticker_file['created'] = datetimes
ticker_file = ticker_file.drop('created_utc','columns')

In [11]:
#Reset indexing
t = ticker_file.copy()
t = t.reset_index()

In [12]:
#Determine which posts have multiple positions in a single post
pos_num = []
for i in t['regexed_combined']:
    if len(i) > 1:
        pos_num.append(len(i))
    else: pos_num.append(1)
#print(pos_num)
total = 0
for i in pos_num:
    total += i

#Expand nested lists into one comprehensive list of positions
singularpositionlist = []
for i in ticker_file['regexed_combined']:
    if len(i) > 1:
        for j in i:
            singularpositionlist.append(j)
    else: singularpositionlist.append(i[0])

In [13]:
#Assign new df with polished open date and close date for 1 position per row
#Adjust dates of posts created and expiry dates to only reflect trading days
output = pd.DataFrame()
open_price, close_price, authors = [],[],[]
openday, closeday = 0,0
i = 0

#If post is made on weekend, use the previous day's price
for dates in ticker_file['created']:
    openday = datetime.strptime(dates,'%Y-%m-%d-%w')
    if openday.weekday() == 5:
        openday += timedelta(days=-1)
    elif openday.weekday() == 6:
        openday += timedelta(days=-2)

#Extend open_date column to reflect multiple positions in one post by duplication
    for j in range(pos_num[i]):
        open_price.append(datetime.strftime(openday,'%Y-%m-%d'))
    i+=1

i=0
for author in ticker_file['author']:
    for j in range(pos_num[i]):
        authors.append(author)
    i+=1

for position_list in ticker_file['regexed_combined']:
    for position in position_list:
        position[3] = datetime.strptime(position[3], '%Y-%m-%d-%w')
        
        #Error catching only
        if position[3].weekday() == 5:
            position[3] += timedelta(days=-1)
        elif position[3].weekday() == 6:
            position[3] += timedelta(days=-2)
        
        close_price.append(datetime.strftime(position[3],'%Y-%m-%d'))
output['open_date'] = open_price
output['close_date'] = close_price
output['positions'] = singularpositionlist
output['author'] = authors

In [14]:
#Pull stock open and close prices for all positions
#Store open in data1, close in data2
data1 = pd.DataFrame()
data2 = pd.DataFrame()
temp = pd.DataFrame()
ticker, start_date, end_date = '', '', ''

for index, row in output.iterrows():
    if row['positions'][0].startswith('$'):
        ticker = row['positions'][0][1:]
    else: ticker = row['positions'][0]
    
    start_date = datetime.strptime(row['open_date'],'%Y-%m-%d')
    end_date = datetime.strptime(row['close_date'],'%Y-%m-%d')
    
    counter = 0
    while (True):
        try:
            if counter > 3: break
            counter+=1
            temp = get_data(ticker,start_date=start_date,end_date=start_date+timedelta(days=1))
            temp['outind'] = index
            data1 = data1.append(temp)
            break
        except KeyError:
            start_date += timedelta(days=-1)
        except:
            break
    time.sleep(.5)
    
    counter = 0
    while (True):
        try:
            if counter > 3: break
            counter+=1
            temp = get_data(ticker,start_date=end_date,end_date=end_date+timedelta(days=1))
            temp['outind'] = index
            data2 = data2.append(temp)
            break
        except KeyError:
            end_date += timedelta(days=1)
        except:
            break
    time.sleep(.5)

KeyboardInterrupt: 

In [ ]:
#Reset indices on open and close df, join to make a cohesive open/close price df
data1 = data1.set_index('outind')
data2 = data2.set_index('outind')
data1 = data1.join(data2, lsuffix='_open', rsuffix-'_close')

In [ ]:
#combine open/close prices with positions and dates, rename columns and drop duplicates
combined = data1.join(output)
combined = combined[['open_date','close_date','positions','ticker_open','open_open','close_close']]
combined = combined.rename(columns={'ticker_open':'ticker','open_open':'open_price','close_close':'close_price'})
combined = combined[~combined.index.duplicated(keep='first')]

In [ ]:
#Calculate price delta over option lifetime, insert new column
combined = combined.fillna(-1)
price_delta = []
for index, row in combined.iterrows():
    if row['close_price'] > -1:
        price_delta.append(row['close_price']-row['open_price'])
    else:
        price_delta.append(0)
combined['price_delta'] = price_delta

In [ ]:
#Determine sentiment over option lifetime, insert new column
sentiment = []
for index, row in combined.iterrows():
    if row['positions'][2] == 'c' or row['positions'][2] == 'C':
        if row['price_delta'] > 0:
            sentiment.append('Correct')
        elif row['price_delta'] < 0:
            sentiment.append('Not Correct')
        else:
            sentiment.append('UNKNOWN')
    else:
        if row['price_delta'] < 0:
            sentiment.append('Correct')
        elif row['price_delta'] > 0:
            sentiment.append('Not Correct')
        else:
            sentiment.append('UNKNOWN')
combined['sentiment'] = sentiment

In [ ]:
#Determine moneyness of option at expiry, add new column
#Negative values are out of the money, positive are in the money
moneyness = []
for index, row in combined.iterrows():
    if row['close_price'] == -1: moneyness.append(0)
    else:
        if row['positions'][2] == 'c' or row['positions'][2] == 'C':
            moneyness.append(float(row['close_price'])-float(row['positions'][1]))
        else:
            moneyness.append(float(row['positions'][1])-float(row['close_price']))
combined['moneyness'] = moneyness

In [ ]:
#Clean up final version of yahoo_fin dataframe
final_copy = combined.copy()
final_copy = final_copy[['positions','ticker','open_date','open_price','close_date','close_price','price_delta','sentiment','moneyness']]
final_copy = final_copy.dropna(subset=['open_price'])
final_copy = final_copy.reset_index()
final_copy = final_copy.drop('index','columns')

In [ ]:
#Save final to csv
f = open('yahoo_fin.csv','w',newline='')
final_copy.to_csv(f)
f.close()